In [18]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('data-2021/Age & sex.csv')

# Keep only the specified columns
df_filtered = df[['COL0', 'COL34', 'COL67']]

# Keep only rows where COL0 begins with '3520'
df_final = df_filtered[df_filtered['COL0'].astype(str).str.startswith('3520')]

# Rename the columns
df_final = df_final.rename(columns={
    'COL0': 'dauid',
    'COL34': 'male', 
    'COL67': 'female'
})

# Convert to numeric and calculate total population
df_final['male'] = pd.to_numeric(df_final['male'], errors='coerce')
df_final['female'] = pd.to_numeric(df_final['female'], errors='coerce')
df_final['total'] = df_final['male'] + df_final['female']

# Convert to relative percentages in decimal format
df_final['male'] = df_final['male'] / df_final['total']
df_final['female'] = df_final['female'] / df_final['total']

# Drop the total column as it's no longer needed
df_final = df_final.drop('total', axis=1)

df_final.head()

,dauid,male,female
239,3520,0.483426,0.516574
240,35200002,0.492537,0.507463
241,35200003,0.523810,0.476190
242,35200004,0.492754,0.507246
243,35200005,0.457831,0.542169


In [22]:
pp = pd.read_csv('data-2021/pop-points.csv')

# Count the number of points for each unique dauid
pp_summary = pp.groupby('dauid').size().reset_index(name='count')

# Join with df_final to get the male/female percentages
pp_with_percentages = pp_summary.merge(df_final, on='dauid', how='left')

# Calculate male and female counts by applying the percentages and round to nearest integer
pp_with_percentages['male_count'] = (pp_with_percentages['count'] * pp_with_percentages['male']).round()
pp_with_percentages['female_count'] = (pp_with_percentages['count'] * pp_with_percentages['female']).round()

# Fill NaN values with 0 before converting to int
pp_with_percentages['male_count'] = pp_with_percentages['male_count'].fillna(0).astype(int)
pp_with_percentages['female_count'] = pp_with_percentages['female_count'].fillna(0).astype(int)

pp_with_percentages.head()

,dauid,count,male,female,male_count,female_count
0,35200002,33,0.492537,0.507463,16,17
1,35200003,31,0.523810,0.476190,16,15
2,35200004,34,0.492754,0.507246,17,17
3,35200005,41,0.457831,0.542169,19,22
4,35200006,52,0.519231,0.480769,27,25


In [28]:
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Create a list to store the gender assignments
gender_assignments = []

# For each dauid, randomly assign genders based on the counts
for dauid in pp_with_percentages['dauid']:
    male_count = pp_with_percentages[pp_with_percentages['dauid'] == dauid]['male_count'].iloc[0]
    female_count = pp_with_percentages[pp_with_percentages['dauid'] == dauid]['female_count'].iloc[0]
    total_count = male_count + female_count
    
    # Create gender array: 1 for male, 0 for female
    genders = np.concatenate([np.ones(male_count), np.zeros(female_count)])
    
    # Randomly shuffle the genders
    np.random.shuffle(genders)
    
    # Get the points for this dauid
    points_for_dauid = pp[pp['dauid'] == dauid]
    
    # Assign genders to points (truncate if needed)
    for i, (idx, row) in enumerate(points_for_dauid.iterrows()):
        if i < len(genders):
            gender_assignments.append({'index': idx, 'gender': 'm' if genders[i] == 1 else 'f'})

# Create a DataFrame from assignments and merge with pp
assignments_df = pd.DataFrame(gender_assignments).set_index('index')
pp_with_gender = pp.join(assignments_df)

pp_with_gender.head()

# Save pp_with_gender to CSV
pp_with_gender.to_csv('data-2021/pop-points-with-gender.csv', index=False)
